In [1]:
# !pip install hdfs
# ref : https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings

In [2]:
from hdfs import InsecureClient
import requests
import json
import datetime as dt


## hdfs로 부터 읽기

In [2]:

# https://docs.python.org/ko/3/library/stdtypes.html#bytes-and-bytearray-operations


## hdfs에 쓰기

In [3]:
# 폴더 전송


## hdfs에 수정하기

## hdfs 권한 수정

## hdfs 삭제

False

## REST_API로 데이터를 호출해 HDFS에 저장

In [3]:
def executeRestApi(method, url, headers, params):  
    
    # raise Exception('응답코드 : 500')  # 예외 테스트
    # err_num = 10/0 # 예외 테스트
    if method == "get":
        res = requests.get(url, params=params, headers=headers)
    else:
        res = requests.post(url, data=params, headers=headers)

    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))
       
    return res.text



### 기준일자 함수

In [4]:
def cal_std_day(befor_day):   


### logger

In [4]:
import logging


In [5]:
# 이렇게 문자열로 찍을 수도 있음
# json양식으로 log를 저장하면 다음에 읽어와서 처리하기도 편하다.



### api 호출

In [6]:
url = 
service_key = 
file_dir = 




## 웹크롤링 hdfs 저장

In [7]:
# pip install BeautifulSoup4
from bs4 import BeautifulSoup

